In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
import numpy as np

/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
#Trend-Following Strategy
#Defining indicators 
def EMA(array, n=20):
    """
    Expotential Moving Average
    from n previous periods
    """
    return pd.Series(array).ewm(span=n, adjust=False).mean()

def MACD(array):
    """MACD Indicator"""
    EMA12 = pd.Series(array).ewm(span=12, adjust=False).mean()
    EMA26 = pd.Series(array).ewm(span=26, adjust=False).mean()
    macd = EMA12 - EMA26
    signal = macd.ewm(span=9, adjust=False).mean()
    return macd, signal 

def ADX(timeSeriesData, n=14):
    """
    Find ADX of time series data
    Assuming input timeSeriesData is a pandas dataframe containing columns High, Low, Close
    with number of periods default to 14
    """
    
    if (not ('High' in timeSeriesData.columns) 
        and ('Low' in timeSeriesData.columns) 
        and ('Close' in timeSeriesData.columns)):
        raise ValueError("Input does not contain necessary data")
        
    #Find Average True Range
    df = timeSeriesData.copy()
    df['TR'] = np.maximum(df['High'] - df['Low'], 
                           np.abs(df['High'] - df['Close'].shift(1)), 
                           np.abs(df['Low'] - df['Close'].shift(1)))

    df['ATR'] = df['TR'].ewm(alpha=1/n, adjust=False).mean()

    #Find Directional Movement
    df['H-pH'] = df['High'] - df['High'].shift(1)
    df['pL-L'] = df['Low'].shift(1) - df['Low']

    df['+DM'] = np.where((df['H-pH'] > df['pL-L']) & (df['H-pH'] > 0), df['H-pH'], 0.0)
    df['-DM'] = np.where((df['pL-L'] > df['H-pH']) & (df['pL-L'] > 0), df['pL-L'], 0.0)

    # #Find Directional Index
    df['+DI'] = df['+DM'].ewm(alpha=1/n, adjust=False).mean()/df['ATR'] * 100
    df['-DI'] = df['-DM'].ewm(alpha=1/n, adjust=False).mean()/df['ATR'] * 100

    # #Find ADX
    df['DX'] = np.abs(df['+DI'] - df['-DI'])/(df['+DI'] + df['-DI']) * 100
    df['ADX'] = df['DX'].ewm(alpha=1/n, adjust=False).mean()

    return df['ADX']

class TrendFollowing(Strategy):
    
    #Parameters need to be optimized
    adx_threshold = 25
    trade_size = 0.98
    tp_ratio = 0.05
    sl_ratio = 0.02
    
    def init(self):
        self.macd, self.macd_sig = self.I(MACD, self.data.Close)
        self.adx = self.I(ADX, self.data.df)
        self.ema30 = self.I(EMA, self.data.Close, 30)

    def next(self):
        price = self.data.Close[-1]
        if (self.macd > 0 and 
            crossover(self.macd, self.macd_sig) and 
            self.adx > self.adx_threshold):
            
            #Close any short position and buy
            if self.position.is_short:
                self.position.close()
            self.buy(tp=(1+self.tp_ratio)*price, sl=(1-self.sl_ratio)*price, size=self.trade_size)
            
        elif (self.macd < 0 and
              crossover(self.macd_sig, self.macd) and 
              self.adx > self.adx_threshold):
            
            #Close any long position and sell
            if self.position.is_long:
                self.position.close()
            self.sell(tp=(1-self.tp_ratio)*price, sl=(1+self.sl_ratio)*price, size=self.trade_size)
        
        #Closing position logic
        if self.position.is_long:
            if (price < 0.99 * self.ema30[-1] and
                self.macd < 0):
                self.position.close()
        if self.position.is_short:
            if (price > 1.01 * self.ema30[-1] and
                self.macd > 0):
                self.position.close()            

            

In [3]:
#Backtest on BTC 15 minutes data for the past 3 month
symbol = "BTC"
interval = "15m"
df = pd.read_csv(f'data/{symbol}_{interval}.csv', index_col=0, parse_dates=True)
display(df)

,Open,High,Low,Close,Volume
date,,,,,
2024-05-01 00:00:00+00:00,60610.00,60991.0,60541.35,60541.35,160.993979
2024-05-01 00:15:00+00:00,60743.00,60875.0,60322.44,60370.84,116.953860
2024-05-01 00:30:00+00:00,60367.51,60581.0,60015.99,60291.27,192.209682
2024-05-01 00:45:00+00:00,60294.50,60550.0,60115.63,60173.03,195.513673
2024-05-01 01:00:00+00:00,60175.44,60475.0,59803.38,59828.94,261.251889
...,...,...,...,...,...
2024-08-01 22:45:00+00:00,65323.12,65516.0,65076.70,65116.22,316.639894
2024-08-01 23:00:00+00:00,65116.60,65346.0,64920.93,65030.00,277.887537
2024-08-01 23:15:00+00:00,65030.00,65194.0,64934.98,65041.07,98.133052


In [129]:
bt = Backtest(df, TrendFollowing, cash=1000000, commission=0.001)
stats = bt.optimize(
    method='grid',
    adx_threshold = range(25, 40, 3),
    trade_size = [0.33, 0.49, 0.99],
    tp_ratio = [0.04, 0.05, 0.06, 0.1],
    sl_ratio = [0.01, 0.02, 0.03, 0.1],
    maximize='Return [%]'
)
print(stats)
print(stats._strategy)
print(stats._trades)
bt.plot(filename=f'backtest_results/{symbol}_{interval}_plot.html')

/opt/anaconda3/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/8 [00:00<?, ?it/s]

Start                     2024-05-01 00:00...
End                       2024-08-01 23:45...
Duration                     92 days 23:45:00
Exposure Time [%]                   23.790323
Equity Final [$]                1232502.45673
Equity Peak [$]                 1232502.45673
Return [%]                          23.250246
Buy & Hold Return [%]                7.840641
Return (Ann.) [%]                  111.728514
Volatility (Ann.) [%]                59.85909
Sharpe Ratio                         1.866525
Sortino Ratio                       11.496677
Calmar Ratio                        15.250826
Max. Drawdown [%]                   -7.326063
Avg. Drawdown [%]                   -1.519498
Max. Drawdown Duration       44 days 02:30:00
Avg. Drawdown Duration        3 days 07:28:00
# Trades                                   34
Win Rate [%]                        29.411765
Best Trade [%]                       6.105556
Worst Trade [%]                     -1.153549
Avg. Trade [%]                    

/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(resample_rule, label='left')
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing val

GridPlot(id='p19653', ...)

In [4]:
#Backtest on SOL 15 minutes data for the past 3 month
symbol = "SOL"
interval = "15m"
df = pd.read_csv(f'data/{symbol}_{interval}.csv', index_col=0, parse_dates=True)
display(df)

,Open,High,Low,Close,Volume
date,,,,,
2024-05-01 00:00:00+00:00,126.63,127.54,126.43,126.43,9480.350219
2024-05-01 00:15:00+00:00,126.47,127.36,125.68,125.92,18365.765105
2024-05-01 00:30:00+00:00,125.92,126.13,124.44,125.00,31545.380961
2024-05-01 00:45:00+00:00,124.99,125.88,124.75,125.04,8946.268212
2024-05-01 01:00:00+00:00,125.02,125.78,124.38,124.55,12497.727483
...,...,...,...,...,...
2024-08-01 22:45:00+00:00,167.59,168.13,166.95,167.31,8411.162567
2024-08-01 23:00:00+00:00,167.31,168.13,166.97,167.26,8852.029155
2024-08-01 23:15:00+00:00,167.27,167.56,166.82,167.11,8739.188324


In [131]:
bt = Backtest(df, TrendFollowing, cash=1000000, commission=0.001)
stats = bt.optimize(
    method='grid',
    adx_threshold = range(25, 40, 3),
    trade_size = [0.33, 0.49, 0.99],
    tp_ratio = [0.04, 0.05, 0.06, 0.1],
    sl_ratio = [0.01, 0.02, 0.03, 0.1],
    maximize='Return [%]'
)
print(stats)
print(stats._strategy)
print(stats._trades)
bt.plot(filename=f'backtest_results/{symbol}_{interval}_plot.html')

/opt/anaconda3/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/8 [00:00<?, ?it/s]

Start                     2024-05-01 00:00...
End                       2024-08-01 23:45...
Duration                     92 days 23:45:00
Exposure Time [%]                   29.569892
Equity Final [$]                1387493.16089
Equity Peak [$]                 1387493.16089
Return [%]                          38.749316
Buy & Hold Return [%]               32.262912
Return (Ann.) [%]                  211.502457
Volatility (Ann.) [%]              137.262595
Sharpe Ratio                          1.54086
Sortino Ratio                       10.081473
Calmar Ratio                        18.371026
Max. Drawdown [%]                  -11.512828
Avg. Drawdown [%]                   -1.882531
Max. Drawdown Duration       15 days 16:00:00
Avg. Drawdown Duration        1 days 18:24:00
# Trades                                   83
Win Rate [%]                        48.192771
Best Trade [%]                       4.251718
Worst Trade [%]                     -3.170304
Avg. Trade [%]                    

/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(resample_rule, label='left')
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing val

GridPlot(id='p20103', ...)

In [6]:
#Mean Rversion Strategy
    
def resid(df1, df2):
    hedge_ratio = 0.04346
    hedge_const = 544.50
    #Values are obtained directly from engle-granger procedure
    #See engle-granger.ipynb
    #Trade ETH against BTC: hedge ratio = 0.04346, hedge const = 544.50, 
    
    return df1 - (df2 * hedge_ratio + hedge_const)

class MeanReversion(Strategy):

    Z_entry = 0.5
    Z_exit = 0.1
    
    def init(self):
        self.resid = self.I(resid, self.data.ETH, self.data.BTC)
        
    def next(self):  
        sigma_eq = 134.79
        mu = -9.91
        #Values are obtained directly from engle-granger procedure
        #See engle-granger.ipynb
        #residuals mean = -9.91, sigma_eq = 134.79
        
        if not self.position:
            if self.resid > mu + self.Z_entry * sigma_eq:
                self.sell()
            elif self.resid < mu - self.Z_entry * sigma_eq:
                self.buy()
        elif (self.resid < mu + self.Z_exit * sigma_eq and
            self.resid > mu - self.Z_exit * sigma_eq):
            self.position.close()


In [7]:
#Mean Reversion backtesting
interval = "daily"
btc_daily = pd.read_csv(f'data/BTC_{interval}.csv', index_col=0, parse_dates=True)
eth_daily = pd.read_csv(f'data/ETH_{interval}.csv', index_col=0, parse_dates=True)
eth_btc_daily= pd.read_csv(f'data/ETH_BTC_{interval}.csv', index_col=0, parse_dates=True)

#ETH_BTC is used for trading purpose
#ETH and BTC appended individually to the dataframe to caluclate residules for signal generation
df = eth_btc_daily.assign(ETH=eth_daily['Close'], BTC=btc_daily['Close'])

display(df.head())

bt = Backtest(df, MeanReversion, cash=10000, commission=0.001, exclusive_orders=True)
stats = bt.optimize(
    method='grid',
    Z_entry = [x / 100.0 for x in range(30, 150, 10)],
    Z_exit = [x / 100.0 for x in range(1, 30, 3)],
    maximize='Return [%]'
)
print(stats)
print(stats._trades)
print(stats._strategy)
bt.plot(filename=f'backtest_results/mean_reversion_plot.html')

,Open,High,Low,Close,Volume,ETH,BTC
date,,,,,,,
2022-09-01,0.07749,0.07923,0.07682,0.07879,8829.630282,1586.23,20133.65
2022-09-02,0.07877,0.08079,0.07810,0.07895,5352.616359,1575.69,19953.74
2022-09-03,0.07890,0.07895,0.07766,0.07855,2349.062462,1557.70,19835.47
2022-09-04,0.07846,0.07939,0.07836,0.07896,4563.738969,1579.04,20004.73
2022-09-05,0.07904,0.08205,0.07897,0.08172,20628.705177,1618.01,19794.58


/opt/anaconda3/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/8 [00:00<?, ?it/s]

Start                     2022-09-01 00:00:00
End                       2024-08-01 00:00:00
Duration                    700 days 00:00:00
Exposure Time [%]                   72.610556
Equity Final [$]                 27162.366044
Equity Peak [$]                  28213.571994
Return [%]                          171.62366
Buy & Hold Return [%]              -37.783983
Return (Ann.) [%]                   68.252027
Volatility (Ann.) [%]               48.265302
Sharpe Ratio                         1.414101
Sortino Ratio                          4.5005
Calmar Ratio                         5.211141
Max. Drawdown [%]                  -13.097329
Avg. Drawdown [%]                   -2.727702
Max. Drawdown Duration      171 days 00:00:00
Avg. Drawdown Duration       14 days 00:00:00
# Trades                                   28
Win Rate [%]                        92.857143
Best Trade [%]                      11.184875
Worst Trade [%]                     -2.600422
Avg. Trade [%]                    

/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing va

GridPlot(id='p1351', ...)